## Naive Bayes Practice

### 직접 구현 
* 긍정(1), 부정(0)
* 새로운 메일이 왔는데 'happy weekend'가 포함되어 있을 경우, 이 메일은 긍정인가, 부정인가 ? 

In [ ]:
# 토큰화 
# 띄어쓰기로 분리 
# unique한 단어처리 
# Laplace smoothing VS underflow 

In [1]:
import pandas as pd 
import numpy as np 
input_file = pd.read_csv('C:/Users/이상은/naivebayes_example.csv') # 읽어오기 
input_file 

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [20]:
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


In [ ]:
## 토큰 빈도수 및 문서별 토큰수 계산 

In [21]:
# 토큰별 문서 내 빈도수 카운팅 
from collections import defaultdict 
wordfreq = defaultdict(lambda: [0,0]) # lambda : 새로운 단어가 추가될 때 default를 [0,0]으로 하겠다
# [긍정, 부정]
for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:
            wordfreq[word][0] += 1 # 긍정인 경우 0번째 인덱스에 1을 더해라 
        else: 
            wordfreq[word][1] += 1 # 부정인 경우 1번째 인덱스에 1을 더해달라 
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [22]:
# 긍정/부정 빈도수 계산 
긍정전체토큰수 = [] 
부정전체토큰수 = [] 
for key, (cnt1, cnt0) in wordfreq.items(): # cnt1과 cnt0는 wordfreq 안에 있는 list 숫자부분
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정, 전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


In [ ]:
## Training : 토큰별 조건부 확률 계산 

In [24]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [25]:
wordprobs = defaultdict(lambda:[0,0]) #새로운 값이 들어올때마다 0,0으로 reset(default)하겠다
for key, (cnt1, cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1 + 1)/(전체갯수_긍정 + len(wordfreq)) # Laplace 정리 적용 
    wordprobs[key][1] = (cnt0 + 1)/(전체갯수_부정 + len(wordfreq))
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

In [ ]:
## Classify : 신규텍스트가 주어졌을 때 확률 계산  

In [26]:
# 실험으로 happy weekend만 조건부확률 계산해보기! 
import math 
doc = "happy weekend"
tokens = doc.split()
tokens

['happy', 'weekend']

In [28]:
# 초기값을 모두 0으로 처리 
log_prob1 = log_prob0 = 0.0 

# 모든 단어에 대해 반복 
for word, (prob1, prob0) in wordprobs.items(): # prob1, prob2처럼 두개 인수 for문에 돌려도 됨 
    if word in tokens: 
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0) # 요 부분까지는 단어 log적용
log_prob1 += math.log(전체갯수_긍정 / (전체갯수_긍정 + 전체갯수_부정))
log_prob0 += math.log(전체갯수_부정 / (전체갯수_긍정 + 전체갯수_부정)) # 요 부분은 log(0.5)처럼
# 긍정/부정을 계산하기 위함임

prob1 = math.exp(log_prob1)
print('prob1', prob1)
prob0 = math.exp(log_prob0)
print('prob0', prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))

prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%


## Using sklearn

In [29]:
# 불러오기 (위에처럼 input_file로 불러온 파일 이름 지정)
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [30]:
X_train = list(input_file['mail'])
Y_train = list(input_file['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# CountVectorizer 선언 
count_vect = CountVectorizer()
# Fit and transform 
X_train_counts = count_vect.fit_transform(X_train)

# MutinomialNB 선언 and fit 
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [33]:
# 예측 
print(clf.predict(count_vect.transform(["happy weekend"])))

#확률 
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

[1]
[[0.25 0.75]]


In [34]:
# 확률은 부정/긍정 순으로 표시됨  